<a href="https://colab.research.google.com/github/knguyenTexans/knguyen_rdp_code/blob/main/mock_int/DA_Mock_Int_Qs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import pandas as pd
import sqlite3

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created.
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)


In [ ]:
# @title
customer_url = "https://raw.githubusercontent.com/knguyenTexans/knguyen_rdp_code/refs/heads/main/tbl_data/customer.csv"
customer_df = pd.read_csv(customer_url)
pd_to_sqlDB(customer_df,
            table_name='customer',
            db_name='default.db')

attend_url = "https://raw.githubusercontent.com/knguyenTexans/knguyen_rdp_code/refs/heads/main/tbl_data/attend.csv"
attend_df = pd.read_csv(attend_url)
pd_to_sqlDB(attend_df,
            table_name='attend',
            db_name='default.db')

ticket_url = "https://raw.githubusercontent.com/knguyenTexans/knguyen_rdp_code/refs/heads/main/tbl_data/ticket.csv"
ticket_df = pd.read_csv(ticket_url)
pd_to_sqlDB(ticket_df,
            table_name='ticket',
            db_name='default.db')

event_url = "https://raw.githubusercontent.com/knguyenTexans/knguyen_rdp_code/refs/heads/main/tbl_data/event.csv"
event_df = pd.read_csv(event_url)
pd_to_sqlDB(event_df,
            table_name='event',
            db_name='default.db')



You will be writing SQL Queries using the following tables:

### Ticket
*   id
*   customer_id
*   event_id
*   section
*   price
*   purchase_date

### Event
*   id
*   event_date
*   name

### Attend
*   customer_id
*   event_id
*   scan_timestamp

### Customer
*   id
*   email
*   zip


## view tbl data

In [ ]:
customerTbl = sql_query_to_pd("SELECT * FROM customer" , db_name='default.db')
customerTbl

,id,email,zip
0,1,alice.johnson@email.com,10001
1,2,bob.smith@email.com,90210
2,3,carol.white@email.com,60601
3,4,david.brown@email.com,2101
4,5,emma.davis@email.com,94102
5,6,frank.miller@email.com,33101
6,7,grace.wilson@email.com,78701
7,8,henry.moore@email.com,10001
8,9,iris.taylor@email.com,90210
9,10,jack.anderson@email.com,60601


In [ ]:
eventTbl = sql_query_to_pd("SELECT * FROM event" , db_name='default.db')
eventTbl

,id,event_date,name,season_year,opponent
0,1,2023-10-15,Home Game,2023,Warriors
1,2,2023-10-22,Home Game,2023,Heat
2,3,2023-11-05,Home Game,2023,Nets
3,4,2023-11-18,Home Game,2023,Bucks
4,5,2023-12-03,Home Game,2023,Suns
5,6,2023-12-20,Home Game,2023,Celtics
6,7,2024-01-08,Home Game,2024,Nuggets
7,8,2024-01-21,Home Game,2024,76ers
8,9,2024-02-10,Home Game,2024,Clippers
9,10,2024-02-25,Home Game,2024,Mavericks


In [ ]:
attendTbl = sql_query_to_pd("SELECT * FROM attend" , db_name='default.db')
attendTbl

,customer_id,event_id,scan_timestamp
0,1,1,2023-10-15 18:30:00
1,1,2,2023-10-22 18:35:00
2,2,1,2023-10-15 18:45:00
3,2,3,2023-11-05 18:40:00
4,3,2,2023-10-22 18:50:00
5,5,1,2023-10-15 18:55:00
6,5,3,2023-11-05 18:45:00
7,5,6,2023-12-20 19:00:00
8,6,2,2023-10-22 19:05:00
9,1,7,2024-01-08 18:30:00


In [ ]:
ticketTbl = sql_query_to_pd("SELECT * FROM ticket" , db_name='default.db')
ticketTbl

## Basic Queries
1.   Find customers who bought tickets but didn't attend
2.   Calculate revenue by event or section
3.   Find customers who attended multiple events
4.   Identify the most popular events by tickets sold vs actual attendance
5.   Calculate no-show rates
6.   Find events with unsold sections
7.   Analyze purchasing patterns (early bird vs last-minute)



In [ ]:
# 1 Find customers who bought tickets but didn't attend
qry1 = """

"""

qry1_result = sql_query_to_pd(qry1)
qry1_result


In [ ]:
# 2 Calculate revenue by event or section
qry2 = """

"""

qry2_result = sql_query_to_pd(qry2)
qry2_result


In [ ]:
# 3 Find customers who attended multiple events
qry3 = """

"""

qry3_result = sql_query_to_pd(qry3)
qry3_result

In [ ]:
# 4 Identify the most popular events by tickets sold vs actual attendance
qry4 = """

"""

qry4_result = sql_query_to_pd(qry4)
qry4_result

In [ ]:
# 5 Calculate no-show rates
qry5 = """

"""

qry5_result = sql_query_to_pd(qry5)
qry5_result

In [ ]:
# 6 Find events with unsold sections
qry6 = """

"""

qry6_result = sql_query_to_pd(qry6)
qry6_result

In [ ]:
# 7 Analyze purchasing patterns (early bird vs last-minute)
qry7 = """

"""

qry7_result = sql_query_to_pd(qry7)
qry7_result


In [ ]:
# 8 Group customers by spending habits
qry8 = """

"""

qry8_result = sql_query_to_pd(qry8)
qry8_result

## Data Quality Checks


You notice some inconsistencies in your customer data. Writer queries to identiy:

1. Duplicate email addresses
2. Customers with tickets but no email
3. Tickets with invalid prices (<=0 or NULL)
4. Future-dated purchases (purchase_date > today)



In [ ]:
# 1. Duplicate email addresses
dq_qry1 = """

"""
dq_qry1_result = sql_query_to_pd(dq_qry1)
dq_qry1_result


In [ ]:
# 2. Customers with tickets but no email
dq_qry2 = """

"""
dq_qry2_result = sql_query_to_pd(dq_qry2)
dq_qry2_result


In [ ]:
# 3. Tickets with invalid prices (<=0 or NULL)
dq_qry3 = """

"""
dq_qry3_result = sql_query_to_pd(dq_qry3)
dq_qry3_result


In [ ]:
# 4. Future-dated purchases (purchase_date > today)
dq_qry4 = """

"""
dq_qry4_result = sql_query_to_pd(dq_qry4)
dq_qry4_result




# Intermediate Queries



### Sales Performance
> Write a query to find the top 5 games by total ticket revenue, including game date, opponent, total tickets sold, and total revenue.
> > Only include games from the 2024 season.


In [ ]:
## Sales Performance Analysis


### Customer Retention
> Writer a query to identify customers who:
>> 1. Purcahsed tickets in 2023
>> 2. Did NOT purchase any tickets in 2024
>> 3. Show their total 2023 spend and number of tickets purcahsed

In [ ]:
## Customer Retention Analysis

# Daily Running Total Query


> Create a report showing daily ticket sales with:
>> - Date
>> - Daily revenue
>> - Cumulative season revenue
>> - 7-day moving average of daily revenue
>>> Order by date


In [ ]:
## Running Totals

## Congrats you're done

In [ ]:
# @title

# Step 4: Exectue the SQL query
result_df = sql_query_to_pd("SELECT * FROM event" , db_name='default.db')
result_df

,id,event_date,name,season_year,opponent
0,1,2023-10-15,Home Game,2023,Warriors
1,2,2023-10-22,Home Game,2023,Heat
2,3,2023-11-05,Home Game,2023,Nets
3,4,2023-11-18,Home Game,2023,Bucks
4,5,2023-12-03,Home Game,2023,Suns
5,6,2023-12-20,Home Game,2023,Celtics
6,7,2024-01-08,Home Game,2024,Nuggets
7,8,2024-01-21,Home Game,2024,76ers
8,9,2024-02-10,Home Game,2024,Clippers
9,10,2024-02-25,Home Game,2024,Mavericks
